<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#6.1-基于最大间隔分隔数据" data-toc-modified-id="6.1-基于最大间隔分隔数据-1">6.1 基于最大间隔分隔数据</a></span></li><li><span><a href="#6.2-寻找最大间隔" data-toc-modified-id="6.2-寻找最大间隔-2">6.2 寻找最大间隔</a></span><ul class="toc-item"><li><span><a href="#6.2.1-分类器求解优化问题" data-toc-modified-id="6.2.1-分类器求解优化问题-2.1">6.2.1 分类器求解优化问题</a></span></li><li><span><a href="#6.2.2-SVM应用的一般框架" data-toc-modified-id="6.2.2-SVM应用的一般框架-2.2">6.2.2 SVM应用的一般框架</a></span></li></ul></li><li><span><a href="#6.3-SMO高效优化算法" data-toc-modified-id="6.3-SMO高效优化算法-3">6.3 SMO高效优化算法</a></span><ul class="toc-item"><li><span><a href="#6.3.1-Platt的SMO算法" data-toc-modified-id="6.3.1-Platt的SMO算法-3.1">6.3.1 Platt的SMO算法</a></span></li><li><span><a href="#6.3.2-应用简化版SMO算法处理小规模数据集" data-toc-modified-id="6.3.2-应用简化版SMO算法处理小规模数据集-3.2">6.3.2 应用简化版SMO算法处理小规模数据集</a></span></li></ul></li><li><span><a href="#6.4-利用完整Platt-SMO算法加速优化" data-toc-modified-id="6.4-利用完整Platt-SMO算法加速优化-4">6.4 利用完整Platt SMO算法加速优化</a></span></li><li><span><a href="#6.5-在复杂数据上应用核函数" data-toc-modified-id="6.5-在复杂数据上应用核函数-5">6.5 在复杂数据上应用核函数</a></span><ul class="toc-item"><li><span><a href="#6.5.1-利用核函数将数据映射到高维空间" data-toc-modified-id="6.5.1-利用核函数将数据映射到高维空间-5.1">6.5.1 利用核函数将数据映射到高维空间</a></span></li><li><span><a href="#6.5.2-径向基核函数" data-toc-modified-id="6.5.2-径向基核函数-5.2">6.5.2 径向基核函数</a></span></li></ul></li><li><span><a href="#6.6-示例：手写识别问题回顾" data-toc-modified-id="6.6-示例：手写识别问题回顾-6">6.6 示例：手写识别问题回顾</a></span></li><li><span><a href="#6.7-本章小结" data-toc-modified-id="6.7-本章小结-7">6.7 本章小结</a></span></li></ul></div>

# SVM
这里介绍SVM一种最流行的实现，序列最小化优化(Sequential Minimal Optimization, SMO)算法，并介绍如何通过核函数的方式拓展SVM到更多的数据集。
## 6.1 基于最大间隔分隔数据
- 优点：泛化错误率低，计算开销不大，结果易解释
- 缺点：对参数调节和核函数的选择敏感，原始分类器不加修改仅适用于处理二分类问题
- 适用数据类型：数值型和标称性数据

**一些概念：**
- 分隔超平面：将数据集划分开的超平面，若数据集的维度是N，则该超平面的维度是N-1
- 间隔：数据点到分隔面的距离
- 支持向量：离分隔面最近的那些数据点

## 6.2 寻找最大间隔
分隔超平面$W^T X+b$,要计算点A到分隔超平面的距离，必须给出分隔面法线或者垂线长度，该值为$\frac{W^T X+b}
{||W||}$
### 6.2.1 分类器求解优化问题
输入数据给分类器会输出一个类别标签，$f(u)$, 当$u<0$时，输出-1；反之，输出1。当数据点正确分类且距离分隔超平面很远时，$label*(W^TX+b)$会是一个很大的正数。

SVM的目标是找到最小间隔的数据点，得到间隔的最大化：
$$argmax_{w,b}\{min_n(lable(W^T+b))\frac{1}{||W||}\}$$
$$s.t.\quad label(W^TX+b)\quad\geq \quad1$$
引入拉格朗日乘子法，将超平面写成数据点的形式：
$$max_a[\sum_{i=1}^{m}\alpha-\frac{1}{2}\sum_{i,j=1}^{m}label^{(i)}\cdot label^{(i)}\cdot a_i\cdot a_j <x^{(i)}, x^{(j)}>]$$
$$a\geq0,\quad and \sum_{i-1}{m}a_i\cdot label^{(i)}=0$$

这里有一个假设：数据必须100%线性可分，对于有扰动数据点的数据，引入松弛变量来允许数据点可以处于分隔面的错误一侧，这样我们的优化目标保持不变，更新约束规则：
$$s.t.\quad C \geq \alpha \geq 0, \quad and \sum_{i-1}^{m} \alpha _i \cdot label^{(i)}$$

### 6.2.2 SVM应用的一般框架
1. 收集数据：可以使用任意方法
2. 准备数据：需要数值型数据
3. 分析数据：有助于可视化分隔超平面
4. 训练算法：SVM的大部分时间都源自于训练，该过程主要实现两个参数的调优
5. 测试算法：十分简单的计算过程就可以实现
6. 使用算法：几乎所有分类问题都可以使用SVM，值得一提的是，SVM本事是一个二分类器，对于多问题应用SVM需要对代码做一些修改

## 6.3 SMO高效优化算法
训练算法，是对上面两个公式进行求解，一个是最小化目标函数，一个是约束条件。可以使用二次规划求解工具进行求解，一旦得到最优的$\alpha$的值，我们就得到了分隔超平面并能够将数据分类。
### 6.3.1 Platt的SMO算法
SMO算法的工作原理是：每次循环选择两个alpha进行优化处理，一旦找到一对合适的alpha（$\alpha _i, \alpha_j$）那么就增大其中一个同时减小另一个。
**条件:** 这两个alpha必须要在间隔边界之外，另该两个alpha为经过区间化处理或在边界上。

### 6.3.2 应用简化版SMO算法处理小规模数据集
SMO算法的外循环会确定要优化最佳alpha对，而简化版会跳过这一部分，首先在数据集上遍历一个alpha，然后在剩下的alpha集合中随机选择另一个alpha，从而构建alpha对，这里有一点很重要，我们要同时改变两个alpha，可以这么做是因为我们有一个约束：
$$\sum \alpha_i \cdot label^{(i)} = 0$$
接下来，构建一个辅助函数，用于在某个区间范围内随机选择一个整数，同时构建另一个函数，用于数值太大时对其进行调整。

In [7]:
def loadDataSet(fileName):
    '''读取数据，按行从txt中读取，前两列是数据点坐标，最后一列是分类label
    传入：
    fileName：文件路径
    返回：
    dataMat：数据点坐标数据矩阵，m×2
    labelMat：数据类别标签，m×1
    '''
    dataMat, labelMat = [], []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat
def selectJrand(i, m):
    '''返回一个0到m以内，与i不同的整数
    传入：
       i： 第一个alpha值的下标，
       m： 数据量的多少
    返回：
    j：一个0到m，与i不同的整数
    '''
    j = i
    while (j==i):
        j = int(random.uniform(0, m))
    return j
def clipAlpha(aj, H, L):
    '''约束alpha_j的上界和下界
    传入：
    aj： alpha值
    H：上界
    L：下届
    返回：约束后的alpha值
    '''
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [8]:
dataArr, labelArr = loadDataSet('testSet.txt')
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [9]:
from numpy import *
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    '''简单的SMO SVM实现，学习带约束目标的alpha值和偏置
    传入：
    dataMatIn：数据矩阵
    classLabels：类别标签
    C：上界
    toler：容错值
    maxIter：最大迭代数
    返回：
    b：偏置
    alpha：优化目标中的alpha值
    '''
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()    # 类标矩阵转置
    m, n = shape(dataMatrix)
    b = 0
    alphas = mat(zeros((m,1)))   # 学习率
    iter = 0
    while (iter < maxIter):    # 外循环，最大迭代次数
        alphaPairsChanged = 0    # flag，alpha对是否同时改变
        for i in range(m):
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b    # 预测类别
            Ei = fXi - float(labelMat[i])    # 偏差
            #if checks if an example violates KKT conditions
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)    # 随机选择另一个alpha j
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()    # Python非引用复制
                alphaJold = alphas[j].copy()
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H:     # 上下界收敛，继续iterate
                    print "L==H"
                    continue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0: 
                    print "eta>=0"
                    continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta    # 更新alpha j
                alphas[j] = clipAlpha(alphas[j],H,L)    # 约束alpha j 到上下界
                if (abs(alphas[j] - alphaJold) < 0.00001): 
                    print "j not moving enough"
                    continue    # alpha j 更新太少，继续iterate
                #update i by the same amount as j
                #the update is in the oppostie direction
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print "iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print "iteration number: %d" % iter
    return b, alphas

In [10]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 1)


iter: 0 i:0, pairs changed 1
iter: 0 i:3, pairs changed 2
L==H
iter: 0 i:5, pairs changed 3
j not moving enough
j not moving enough
iter: 0 i:18, pairs changed 4
L==H
L==H
j not moving enough
L==H
iter: 0 i:44, pairs changed 5
L==H
L==H
iter: 0 i:54, pairs changed 6
L==H
L==H
L==H
j not moving enough
iteration number: 0
iter: 0 i:0, pairs changed 1
iter: 0 i:3, pairs changed 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:29, pairs changed 3
j not moving enough
j not moving enough
L==H
j not moving enough
iter: 0 i:70, pairs changed 4
j not moving enough
iter: 0 i:77, pairs changed 5
iter: 0 i:82, pairs changed 6
j not moving enough
iteration number: 0
j not moving enough
iter: 0 i:5, pairs changed 1
L==H
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
iter: 0 i:54, pairs changed 2
j not moving enough
iter: 0 i:69, pairs changed 3
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:82, pairs changed 4
j

In [11]:
# 观察alpha矩阵中非零元素
print "not 0 elements in alpha matrix:", alphas[alphas>0]
# 了解支撑向量的数量
print "number of suport vector:", shape(alphas[alphas>0])
# 输出哪些数据点是支撑向量
count = 0
for i in range(100):
    if alphas[i] > 0.0:
        count += 1
        print "suport vector %d: ; x_value: %s; label: %f" % (count, dataArr[i], labelArr[i])

not 0 elements in alpha matrix: [[ 0.08852666  0.01940376  0.02603069  0.00578016  0.16796809  0.02218113
   0.00410569  0.05652747  0.00578016  0.0264563   0.02201125]]
number of suport vector: (1, 11)
suport vector 1: ; x_value: [3.542485, 1.977398]; label: -1.000000
suport vector 2: ; x_value: [3.223038, -0.552392]; label: -1.000000
suport vector 3: ; x_value: [3.457096, -0.082216]; label: -1.000000
suport vector 4: ; x_value: [2.893743, -1.643468]; label: -1.000000
suport vector 5: ; x_value: [5.286862, -2.358286]; label: 1.000000
suport vector 6: ; x_value: [6.543888, 0.433164]; label: 1.000000
suport vector 7: ; x_value: [-0.236713, -5.766721]; label: -1.000000
suport vector 8: ; x_value: [1.966279, -1.840439]; label: -1.000000
suport vector 9: ; x_value: [8.398012, 1.584918]; label: 1.000000
suport vector 10: ; x_value: [8.54562, 2.788799]; label: 1.000000
suport vector 11: ; x_value: [2.912122, -0.202359]; label: -1.000000


## 6.4 利用完整Platt SMO算法加速优化
Platt SMO算法通过一个外循环选择一个alpha值，并且其选择过程会在两种方式之间进行交替：
- 在所有数据集上进行单遍扫描
- 在非边界alpha（不等于边界0或C）中实现单边扫描 建立alpha值的列表，遍历该列表时，会跳过已知不会改变的alpha值

In [12]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))    # m×2的缓存矩阵
def calcEk(oS, k):
    '''计算误差
    传入：
    oS：
    k：
    返回：
    Ek 误差'''
    fXk = float(multiply(oS.alphas, oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek
def selectJ(i, oS, Ei):
    '''选择第二个alpha值，使用内循环启发方法
    传入
    i：
    oS：
    Ei：
    返回
    j：
    Ej：
    '''
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]    # 构造一个非零表，返回的是非零E值对应的alpha值
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):    # 选择具有最大步长的j
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:    # 第一次循环，随机选择一个alpha j的值
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej
def updateEk(oS, k):
    '''计算误差并缓存'''
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]


In [13]:
def innerL(i,oS):
    Ei = calcEk(oS, i)
    if (oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C) or (oS.labelMat[i]*Ei > oS.tol) and\
    (oS.alphas[i] > 0):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[j])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H:
            print "L == H"
            return 0
        eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - \
              oS.X[j,:]*oS.X[j,:].T
        if eta >= 0:
            print "eta>=0"
            return 0    
        
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta    # 更新alpha j
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)    # 约束alpha j 到上下界
        updateEk(oS,j)    # 更新误差缓存，比简单SMO多了这一步
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): 
            print "j not moving enough"
            return 0    # alpha j 更新太少，继续iterate
        #update i by the same amount as j
        #the update is in the oppostie direction
        
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*\
             oS.X[i,:]*oS.X[i,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*\
             oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*\
             oS.X[i,:]*oS.X[j,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*\
             oS.X[j,:]*oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

In [14]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    '''完整版Platt SMO算法，'''
    oS = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while iter < maxIter and (alphaPairsChanged > 0 or entireSet):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i, oS)
            print "fullSet, iter: %d i:%d, pairs changed %d" %(iter, i, alphaPairsChanged)
            iter += 1
        else:
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print "non-bound, iter: %d i:%d, pairs changed %d" %(iter, i, alphaPairsChanged)
            iter += 1
        if entireSet :
            entireSet = False
        elif alphaPairsChanged == 0:
              entireSet = True
              print "iteration number；%d" % iter
        return oS.b, oS.alphas

In [15]:
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L == H
L == H
L == H
L == H
L == H
j not moving enough
L == H
j not moving enough
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
fullSet, iter: 0 i:99, pairs changed 7


In [16]:
def calcWs(alphas, dataArr,classLabels):
    X = mat(dataArr)
    labelMat = mat(classLabels).transpose()
    m, n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*labelMat[i], X[i,:].T)
    return w
ws = calcWs(alphas,dataArr,labelArr)

In [17]:
# 检查分类的正确性
dataMat = mat(dataArr)
for i in range(10):
    print "predict class: %d ;ground truth class: %d" % (dataMat[i] * mat(ws) +b, labelArr[i])

predict class: -1 ;ground truth class: -1
predict class: -1 ;ground truth class: -1
predict class: 1 ;ground truth class: 1
predict class: -1 ;ground truth class: -1
predict class: 0 ;ground truth class: 1
predict class: 0 ;ground truth class: 1
predict class: 1 ;ground truth class: 1
predict class: -1 ;ground truth class: -1
predict class: -1 ;ground truth class: -1
predict class: -2 ;ground truth class: -1


## 6.5 在复杂数据上应用核函数
非线性可分的数据，需要使用核函数将数据转换到分类器理解的形式，本章主要介绍径向基函数。
### 6.5.1 利用核函数将数据映射到高维空间
对数据进行处理，把核函数当作装饰器或者接口，来处理特征空间的映射。SVM优化特别好的地方是，所有运算都能写成内积(inner product)，运用核技巧，将向量内积计算转换为核函数。接下来介绍一个流行的核函数。
### 6.5.2 径向基核函数
径向基函数将向量作为输入，输出是一个标量，其高斯形式是：
$$k(x,y)=(\frac{-||x-y||^2}{2\sigma ^2})$$

In [51]:
from numpy import *
def kernelTrans(X, A, kTup):
    m, n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0] == 'lin':
        K = X*A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K/(-1*kTup[1]**2))
    else:
        raise NameError('Houston We Have a Probem -- That Kernel is not recognized')
    return K

class optStruct_modify:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))
        self.K = mat(zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)

In [52]:
def innerL_modify(i,oS):
    Ei = calcEk(oS, i)
    if (oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C) or (oS.labelMat[i]*Ei > oS.tol) and\
    (oS.alphas[i] > 0):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[j])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H:
            print "L == H"
            return 0
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]
        if eta >= 0:
            print "eta>=0"
            return 0    
        
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta    # 更新alpha j
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)    # 约束alpha j 到上下界
        updateEk(oS,j)    # 更新误差缓存，比简单SMO多了这一步
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): 
            print "j not moving enough"
            return 0    # alpha j 更新太少，继续iterate
        #update i by the same amount as j
        #the update is in the oppostie direction
        
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        b1 = oS.b -Ei - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i]-\
             oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b -Ej - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]-\
             oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

In [53]:
def calcEk_modify(oS, k):
    fXk = float(multiply(oS.alphas, oS.labelMat).T*oS.K[:, k]+oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

In [54]:
def smoP_modify(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    '''完整版Platt SMO算法，'''
    oS = optStruct_modify(mat(dataMatIn), mat(classLabels).transpose(), C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while iter < maxIter and (alphaPairsChanged > 0 or entireSet):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL_modify(i, oS)
            print "fullSet, iter: %d i:%d, pairs changed %d" %(iter, i, alphaPairsChanged)
            iter += 1
        else:
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL_modify(i, oS)
                print "non-bound, iter: %d i:%d, pairs changed %d" %(iter, i, alphaPairsChanged)
            iter += 1
        if entireSet :
            entireSet = False
        elif alphaPairsChanged == 0:
              entireSet = True
              print "iteration number；%d" % iter
        return oS.b, oS.alphas

In [57]:
def testRbf(k1=1.3):
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    b, alphas = smoP_modify(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A>0)[0]
    sVs = dataMat[svInd]
    labelSV = labelMat[svInd]
    print "there are %d Support Vectors" % shape(sVs)[0]
    m, n = shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, dataMat[i,:], ('rbf', k1))
        predict = kernelEval.T*multiply(labelSV, alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print "the training error rate is: %f" % (float(errorCount/m))
    dataArr, labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m, n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, dataMat[i,:], ('rbf', k1))
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print "the training error rate is: %f" % (float(errorCount/m))
    

In [58]:
testRbf()

L == H
fullSet, iter: 0 i:99, pairs changed 73
there are 72 Support Vectors
the training error rate is: 0.000000
the training error rate is: 0.000000


## 6.6 示例：手写识别问题回顾
1. 收集数据：提供文本文件
2. 准备数据：基于二值图像构造向量
3. 分析数据：对图像向量进行目测
4. 训练算法：采用两种不同的核函数，并对RBF采用不同设置来运行SMO算法
5. 测试算法：编写一个函数来测试不同的核函数并计算错误率
6. 使用算法：不打算深入

In [83]:
def img2vector(filename):
    returnVect = zeros((1, 1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect
def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' %(dirName, fileNameStr))
    return trainingMat, hwLabels
def testDigits(kTup=('rbf', 10)):
    dataArr, labelArr = loadImages('trainingDigits')
    b, alphas = smoP_modify(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A > 0)[0]
    sVs = dataMat[svInd]
    labelSV = labelMat[svInd]
    print "there are %d Support Vectors" % shape(sVs)[0]
    m, n = shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, dataMat[i,:], kTup)
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print "the training error rate is: %f" % (float(errorCount)/m)
    dataArr, labelArr = loadImages('testDigits')
    errorCount = 0
    dataMat, labelMat = mat(dataArr), mat(labelArr).transpose()
    m, n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, dataMat[i,:], kTup)
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print "the test error rate is: %f" % (float(errorCount)/m)

In [84]:
testDigits(('rbf', 20))

L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H

## 6.7 本章小结
支持向量机是一种分类器，之所以成为“机”是因为会产生一个二值决策结果，它是一种决策“机”。支持向量机的泛化错误比较低，学习到的结果具有很好的推广性。